In [22]:
import docker
import uuid

def run_in_master(command):
    client = docker.from_env()
    container = client.containers.get('master')
    return container.exec_run(command)

def get_data_from_output_path(path):
    return f"{path}/part-r-00000"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}").output.decode('utf-8')
    print(f"{raw[0:10000]}")
    


# Covid

## Wybór kolumn

In [23]:
covid_01_jar_path = "/data/master_volume/map_reduce_jars/covid_01.jar"
covid_01_input_path = "/datasets/covid-dataset.jsonl"
covid_01_output_path = "/out_covid_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_01_jar_path} {covid_01_input_path} {covid_01_output_path}")
print(res.exit_code == 0)

True


In [26]:
print_hdfs_output(covid_01_output_path)

## Wybór wymiaru czasu

In [21]:
covid_02_jar_path = "/data/master_volume/map_reduce_jars/covid_02.jar"
covid_02_input_path = get_data_from_output_path(covid_01_output_path)
covid_02_output_path = "/out_covid_2" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_02_jar_path} {covid_02_input_path} {covid_02_output_path}")
print(res.exit_code == 0)

True


In [ ]:
print_hdfs_output(covid_02_output_path)

# Steam

## Agregacja danych ze źródeł Steam

In [6]:
steam_01_jar_path = "/data/master_volume/map_reduce_jars/steam_01_combine.jar"
steam_01_input_path = "/datasets/steam-dataset/steam_dataset/appinfo/store_data/steam_store_data.jsonl"
steam_01_input_path2 = "/datasets/steam-dataset/steam_dataset/steamspy/basic/steam_spy_scrap.jsonl"
steam_01_output_path = "/out_steam_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_01_jar_path} {steam_01_input_path} {steam_01_input_path2} {steam_01_output_path}")

print(res)

ExecResult(exit_code=0, output=b"2023-05-07 00:32:56,528 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at master/10.1.2.3:8050\n2023-05-07 00:32:56,722 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.\n2023-05-07 00:32:56,727 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1683384448395_0001\n2023-05-07 00:32:56,998 INFO input.FileInputFormat: Total input files to process : 2\n2023-05-07 00:32:57,049 INFO mapreduce.JobSubmitter: number of splits:6\n2023-05-07 00:32:57,129 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1683384448395_0001\n2023-05-07 00:32:57,129 INFO mapreduce.JobSubmitter: Executing with tokens: []\n2023-05-07 00:32:57,212 INFO conf.Configuration: resource-types.xml not found\n2023-05-07 00:32:57,212 INFO resource.ResourceUtils: Unable

In [14]:
print(res.exit_code)

1


In [8]:
raw = run_in_master(f"hdfs dfs -cat {steam_01_output_path}/part-r-00000").output.decode('utf-8')
print(f"{raw[0:10000]}")

10	{"game_id":10,"name":"Counter-Strike","positive":196557,"negative":5070,"owners":"10,000,000 .. 20,000,000","ccu":12877,"release_date":"2000-11-01"}
1000000	{"game_id":1000000,"name":"ASCENXION","positive":27,"negative":5,"owners":"0 .. 20,000","ccu":0,"release_date":"2021-05-14"}
1000010	{"game_id":1000010,"name":"Crown Trick","positive":3809,"negative":583,"owners":"200,000 .. 500,000","ccu":51,"release_date":"2020-10-16"}
1000030	{"game_id":1000030,"name":"Cook, Serve, Delicious! 3?!","positive":1469,"negative":101,"owners":"50,000 .. 100,000","ccu":54,"release_date":"2020-10-14"}
1000040	{"game_id":1000040,"name":"细胞战争","positive":0,"negative":1,"owners":"0 .. 20,000","ccu":0,"release_date":"2019-03-30"}
1000080	{"game_id":1000080,"name":"Zengeon","positive":1011,"negative":431,"owners":"50,000 .. 100,000","ccu":2,"release_date":"2019-06-24"}
1000100	{"game_id":1000100,"name":"干支セトラ　陽ノ卷｜干支etc.　陽之卷","positive":18,"negative":6,"owners":"0 .. 20,000","ccu":0,"release_date":"2019-01

## Selekcja gier

In [9]:
steam_02_jar_path = "/data/master_volume/map_reduce_jars/steam_02_choose.jar"
steam_02_input_path = steam_01_output_path
steam_02_output_path = "/out_steam_2" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_02_jar_path} {steam_02_input_path} {steam_02_output_path}")

In [10]:
raw = run_in_master(f"hdfs dfs -cat {steam_02_output_path}/part-r-00000").output.decode('utf-8')
print(f"{raw[0:10000]}")

## Wybór pierwszych N gier

In [ ]:
steam_03_jar_path = "/data/master_volume/map_reduce_jars/steam_03.jar"
steam_03_input_path = steam_02_output_path
steam_03_output_path = "/out_steam_3" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_03_jar_path} {steam_03_input_path} {steam_03_output_path}")

## Pobranie danych ze Steam Charts API

In [ ]:
steam_04_jar_path = "/data/master_volume/map_reduce_jars/steam_04.jar"
steam_04_input_path = steam_03_output_path
steam_04_output_path = "/out_steam_4" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_04_jar_path} {steam_04_input_path} {steam_04_output_path}")